
Run this code on google colab to upsample a video using pre-trained Real-ESRGAN 

---


Download this notebook, upload it on google colab and then execute


---



Upload the input LR **mp4** video and weights, preferably in the '/content/' folder.

Download the weights file from here:- https://drive.google.com/drive/folders/16PlVKhTNkSyWFx52RPb2hXPIQveNGbxS

Upload the 'RealESRGAN_x4.pth' file in the '/content/' folder 


---


You must connect the runtime to GPU


---


This is a SISR technique, hence it will upsample single image frame at a time. Real-ESRGAN shows better results on faces than ESRGAN. However, it is quite slow as compared to other SRGAN techniques.

# Importing necessary libraries

In [ ]:
import cv2
import os
import numpy as np
import datetime
from PIL import Image
import torch

# Cloning the Real-ESRGAN github repository 

In [ ]:
!git clone https://github.com/sberbank-ai/Real-ESRGAN

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 42 (delta 11), reused 25 (delta 3), pack-reused 0
Unpacking objects: 100% (42/42), done.


# Installing the required packages


In [ ]:
!pip install -r /content/Real-ESRGAN/requirements.txt

In [ ]:
cd /content/Real-ESRGAN

/content/Real-ESRGAN


# Loading the pre-trained Real-ESRGAN model
Add weights file 'RealESRGAN_x4.pth' as an argument to the function load_weights()

In [ ]:
from realesrgan import RealESRGAN

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = RealESRGAN(device, scale=4)
model.load_weights('/content/RealESRGAN_x4.pth')

# Defining the function to apply video on ESRGAN


In [ ]:
def ESRGAN_on_video(vid_path, kps):

  video = cv2.VideoCapture(vid_path)

  (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

  if int(major_ver)  < 3 :
    fps = int(video.get(cv2.cv.CV_CAP_PROP_FPS))
  else :
    fps = int(video.get(cv2.CAP_PROP_FPS))

  if (fps == 0): 
    print('pls upload a different video')
    exit(0)

  print(fps)

  hop = round(fps/kps)
  if hop == 0:
    print('pls select such custom fps such that fps<=',fps)
    exit(0)

  frame_dir = '/content/framedir'
  os.makedirs(frame_dir)
  output_dir = '/content/outframes'
  os.makedirs(output_dir)

  hop = round(fps / kps)
  
  arr_output_name = []
  currentframe = 0

  while(True):
      ret,frame = video.read()
      if ret:
        if (currentframe % hop == 0): 
          name = frame_dir + '/' + str(int(currentframe/hop)) + '.jpg'
          cv2.imwrite(name, frame)
        image = Image.open(name).convert('RGB')
        sr_image = model.predict(image)
        sr_image.save(output_dir + '/' + str(int(currentframe/hop)) + '.jpg')
        arr_output_name.append(output_dir + '/' + str(int(currentframe/hop)) + '.jpg')
        currentframe += 1
      else:
          break

  video.release()

  img = cv2.imread(output_dir + '/0.jpg')
  height, width, layers = img.shape
  size = (width,height)
          
  out = cv2.VideoWriter('/content/ESRGAN_output_vid.mp4',cv2.VideoWriter_fourcc(*'DIVX'), kps , size)

  for i in range(len(arr_output_name)):
      out.write(cv2.imread(arr_output_name[i]))
  out.release()


# Apply the pre-trained generator on custom video
Add the path to your uploaded video (str) and fps (int) as argument to function ESRGAN_on_video()



In [ ]:
%%time
ESRGAN_on_video('/content/assault_part_2_shortened.mp4', 5)

30


# Removing the stored input and output frames

In [ ]:
for image in os.listdir('/content/framedir'):
  os.remove('/content/framedir' + '/' + image)
os.rmdir('/content/framedir')
for image in os.listdir('/content/outframes'):
  os.remove('/content/outframes' + '/' + image)
os.rmdir('/content/outframes')